In [4]:
import requests
import bs4
import json
import time

In [2]:
#base url that is used in all of the pages, format inbetween curly braces with relative path.
base_url = "https://www.nytimes.com/books/best-sellers/{}/"

In [3]:
# the part of the url that points to the exact web page.
relative_url = ""
# string list of relative url paths
urls = []
urls.insert(0, "combined-print-and-e-book-fiction")
urls.insert(1, "combined-print-and-e-book-nonfiction")
urls.insert(2, "hardcover-fiction")
urls.insert(3, "hardcover-nonfiction")
urls.insert(4, "trade-fiction-paperback")
urls.insert(5, "paperback-nonfiction")
urls.insert(6, "advice-how-to-and-miscellaneous")
urls.insert(7, "childrens-middle-grade-hardcover")
urls.insert(8, "picture-books")
urls.insert(9, "series-books")
urls.insert(10, "young-adult-hardcover")

In [4]:
# = note
#* = soup method call

# individual page header, included is when the week finishes.
#* soup.select(".css-68u1tu")

# title
#* soup.select("ol > li > article > div > a > h3")

# number of weeks in list
#* soup.select("ol > li > article > div > a > p")[0].text

# author
#* soup.select("ol > li > article > div > a > p")[1].text

# publisher
#* soup.select("ol > li > article > div > a > p")[2].text

# description
#* soup.select("ol > li > article > div > a > p")[3].text

# links to buy book
#* soup.select("ol > li > article > div > div > div > div > ul > li > a")[0]['href']

# link text title as they appear
#* soup.select("ol > li > article > div > div > div > div > ul > li > a")[0].text

# image url
#* soup.select("ol > li > article > footer > div > a > img")[0]['src']

In [11]:
# iterate through all list items using the above soup method calls and save as a json file format.

#create the object list we will use to append each object of the key_value_pairs to:
key_value_pairs_list = []

# create a header to use with requests, so the web page does not block us as a robot.
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

for string in urls:
    print("Executing..." + string)
    #create the master_list we will use to append the each list of key_value_pairs to:
    master_list = []
    
    #set the relative_url_string
    relative_url = string
    
    # open portal to url
    res = requests.get(base_url.format(relative_url), headers=headers)
    
    # create the soup
    soup = bs4.BeautifulSoup(res.text, "lxml")
    
    #get the catergory header for the list of books.
    category_header = soup.select(".css-68u1tu")[0].text

    #get all the books in the list. this class call points to the first ordered list in the web-page.
    list_items = soup.select(".css-12yzwg4 > li > article")

    #int value used for setting the rank of the book in the list.
    list_rank = 0

    #for each item in the list, exctract the relevant information:
    for item in list_items:
        try:
            #set the book rank
            list_rank = list_rank + 1
            
            #get book title.
            if (item.select("div > a > h3") != []):
                title = item.select("div > a > h3")[0].text
            #get author, number of weeks in the list, publisher and description.
            if (item.select("div > a > p") != []):
                weeks_in_list = item.select("div > a > p")[0].text
                author = item.select("div > a > p")[1].text
                publisher = item.select("div > a > p")[2].text
                description = item.select("div > a > p")[3].text
            #get the store text and link url to buy book.
            if (item.select("div > div > div > div > ul > li > a") != []):
                #store each set of store name and adresses for each book in an array list.
                # create the array
                purchase_websites_list = []
                                
                for link in (item.select("div > div > div > div > ul > li > a")):
                    #create review_quotes text initiated to 'No reviews found'
                    #(in case there are none).
                    #review_quotes = "No reviews found :("
                    # get links and store name
                    website_name = link.text
                    website_link = link['href']
                        
                    # create the object
                    purchase_websites_object = {}
                    #set the key value pairs
                    purchase_websites_object["storeName"] = website_name
                    purchase_websites_object["webAdress"] = website_link
                    # add the key value pairs to the array
                    purchase_websites_list.append(purchase_websites_object)
                    
            #get the image url for each book.
            if (item.select("footer > div > a > img")):
                image_url = item.select("footer > div > a > img")[0]['src']
        
            #create a key_value_pairs list object
            key_value_pairs = {}
        
            # set the key value pair attributes.
            key_value_pairs['title'] = title
            key_value_pairs['author'] = author
            key_value_pairs['description'] = description
            key_value_pairs['publisher'] = publisher
            key_value_pairs['weeks_in_list'] = weeks_in_list
            key_value_pairs['image_url'] = image_url
            key_value_pairs['list_rank'] = list_rank
            key_value_pairs['purchase_websites'] = purchase_websites_list
        
            #append the key_values_pairs object to the key_value_pairs list 
            key_value_pairs_list.append(key_value_pairs)
        
        except AttributeError as ex:
            print('Error:', ex)
        
    # create a new master object to append each key_values_pair list 
    # along with the asscociated category header.
    master_object = {}

    #set the category header in the newly created master_object
    master_object["categoryHeader"] = category_header
    master_object["listItems"] = key_value_pairs_list

    #append master_object to the master_list
    master_list.append(master_object)
    
    # create a new list for the next category to save to.
    key_value_pairs_list = []
    
    #create the object json will convert
    convert_object = {}
    convert_object['contentList'] = master_list

    #convert master_list into a json file format and save file:
    f = open(string + '_bestSellingBooks.json', 'w')
    f.write(json.dumps(convert_object))
    f.close()
    print("finished and saved as json file..." + string + '_bestSellingBooks.json')
    print("Waiting here for 15 seconds to avoid bot detection")
    time.sleep(15)
 
 # after loop 
print("Loop Finished")

Executing...combined-print-and-e-book-fiction
finished and saved as json file...combined-print-and-e-book-fiction_bestSellingBooks.json
Waiting here for 15 seconds to avoid bot detection
Executing...combined-print-and-e-book-nonfiction
finished and saved as json file...combined-print-and-e-book-nonfiction_bestSellingBooks.json
Waiting here for 15 seconds to avoid bot detection
Executing...hardcover-fiction
finished and saved as json file...hardcover-fiction_bestSellingBooks.json
Waiting here for 15 seconds to avoid bot detection
Executing...hardcover-nonfiction
finished and saved as json file...hardcover-nonfiction_bestSellingBooks.json
Waiting here for 15 seconds to avoid bot detection
Executing...trade-fiction-paperback
finished and saved as json file...trade-fiction-paperback_bestSellingBooks.json
Waiting here for 15 seconds to avoid bot detection
Executing...paperback-nonfiction
finished and saved as json file...paperback-nonfiction_bestSellingBooks.json
Waiting here for 15 seconds